In [1]:
# Set up dependencies
import pandas as pd
from pprint import pprint 
import time
import requests as req
from bs4 import BeautifulSoup as bs
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
# Set up chromedriver and browser
executable_path = {"executable_path": ChromeDriverManager().install()}
browser = Browser("chrome", **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 91.0.4472
[WDM] - Get LATEST driver version for 91.0.4472


[WDM] - Driver [C:\Users\rtear\.wdm\drivers\chromedriver\win32\91.0.4472.19\chromedriver.exe] found in cache


## Mars News Scraping 

In [3]:
# set up url for Mars News
url = 'https://mars.nasa.gov/news/'
browser.visit(url)
time.sleep(3)
html = browser.html

In [4]:
# parse with BeautifulSoup
soup = bs(html, 'html.parser')

In [5]:
# find the latest news article
slides = soup.find_all('li', class_='slide')
content_title = slides[0].find('div', class_ = 'content_title')
news_title = content_title.text.strip()
article_teaser_body = slides[0].find('div', class_ = 'article_teaser_body')
news_article = article_teaser_body.text.strip()

In [6]:
# print article title and  paragraph text
pprint(news_title)
pprint(news_article)

"NASA's InSight Mars Lander Gets a Power Boost"
('The spacecraft successfully cleared some dust off its solar panels, helping '
 'to raise its energy and delay when it will need to switch off its science '
 'instruments.')


## Mars Fact Scraping

In [7]:
# set up url for Mars Facts
facts_url = "https://space-facts.com/mars/"
browser.visit(facts_url)
time.sleep(3)

In [8]:
# Parse facts url to find tables
facts_table = pd.read_html(facts_url)
mars_table = facts_table[0] 

In [9]:
# rename columns
mars_table = mars_table.rename(columns = {0:'Description', 1:'Mars'})
mars_table['Description'] = mars_table['Description'].str.replace(':','')
mars_table

,Description,Mars
0,Equatorial Diameter,"6,792 km"
1,Polar Diameter,"6,752 km"
2,Mass,6.39 × 10^23 kg (0.11 Earths)
3,Moons,2 (Phobos & Deimos)
4,Orbit Distance,"227,943,824 km (1.38 AU)"
5,Orbit Period,687 days (1.9 years)
6,Surface Temperature,-87 to -5 °C
7,First Record,2nd millennium BC
8,Recorded By,Egyptian astronomers


In [10]:
# convert data to html
html_table = mars_table.to_html(table_id="html_tbl_css",justify='left',index=False)

In [11]:
# print table
pprint(html_table)

('<table border="1" class="dataframe" id="html_tbl_css">\n'
 '  <thead>\n'
 '    <tr style="text-align: left;">\n'
 '      <th>Description</th>\n'
 '      <th>Mars</th>\n'
 '    </tr>\n'
 '  </thead>\n'
 '  <tbody>\n'
 '    <tr>\n'
 '      <td>Equatorial Diameter</td>\n'
 '      <td>6,792 km</td>\n'
 '    </tr>\n'
 '    <tr>\n'
 '      <td>Polar Diameter</td>\n'
 '      <td>6,752 km</td>\n'
 '    </tr>\n'
 '    <tr>\n'
 '      <td>Mass</td>\n'
 '      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n'
 '    </tr>\n'
 '    <tr>\n'
 '      <td>Moons</td>\n'
 '      <td>2 (Phobos &amp; Deimos)</td>\n'
 '    </tr>\n'
 '    <tr>\n'
 '      <td>Orbit Distance</td>\n'
 '      <td>227,943,824 km (1.38 AU)</td>\n'
 '    </tr>\n'
 '    <tr>\n'
 '      <td>Orbit Period</td>\n'
 '      <td>687 days (1.9 years)</td>\n'
 '    </tr>\n'
 '    <tr>\n'
 '      <td>Surface Temperature</td>\n'
 '      <td>-87 to -5 °C</td>\n'
 '    </tr>\n'
 '    <tr>\n'
 '      <td>First Record</td>\n'
 '      <td>2nd millennium 

## Mars Image Scraping

In [12]:
# Set up url for Mars Image
base_url = "https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/"
image_url = base_url + "index.html"
browser.visit(image_url)
time.sleep(3)
html = browser.html

In [13]:
# parse with BeautifulSoup
soup = bs(html, "html.parser")

In [14]:
# scrape the URL
feature_url = soup.find('a', class_='showimg fancybox-thumbs')['href']
feature_image_url = base_url + feature_url

In [15]:
# print url
pprint(feature_image_url)

'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/image/featured/mars1.jpg'


## Mars Hemispheres Scraping

In [16]:
# Set up url for Mars Hemisphere
hemi_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(hemi_url)
html = browser.html

In [17]:
# beautifulSoup
soup = bs(html, "html.parser")

In [18]:
# retrievethe hemi info
hemi_items = soup.find_all('div', class_='item')

# set base url
hemi_base = 'https://astrogeology.usgs.gov'

In [19]:
# List for hemi info storage
hemi_image_urls = []

# Loop for hemisphere info
for item in hemi_items:

    # store items
    title = item.find('h3').text

    # link to hemi image
    hemi_link = item.find('a', class_='itemLink product-item')['href']

    # Set browser link
    browser.visit(hemi_base + hemi_link)

    # Set up url for Mars hemi image
    hemi_html = browser.html
    hemi_soup = bs(hemi_html, 'html.parser')

    # URL for hemi images
    hemi_url = hemi_base + hemi_soup.find('img', class_='wide-image')['src']

    # append title and image url to dictionary
    hemi_image_urls.append({
        "title": title,
         "img_url": hemi_url
         })

In [20]:
# print the hemispheres images
pprint(hemi_image_urls)

[{'img_url': 'https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'https://astrogeology.usgs.gov/cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'img_url': 'https://astrogeology.usgs.gov/cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'img_url': 'https://astrogeology.usgs.gov/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'}]


In [21]:
# close the browser
browser.quit()